**CQACO: Constrained Qunatum Ant Colony Optimization**
QACO is an hybrid quantum algorithm  based on the classical Ant Colony Optimization algorithm to produce
approximate solutions for NP-hard problems, in particular optimization problems. The constrained version of QACO is adapt to problems that have constrained solutions, which specifically means that the numaber of 1's is fixed, a priori. :

**Importing libraries and qiskit environment**

In [ ]:
# !pip install "qiskit==0.46.0" "qiskit-aer==0.13.3"

In [4]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit import transpile

print("✅ Qiskit importato correttamente!")

✅ Qiskit importato correttamente!


In [5]:
qc = QuantumCircuit(1, 1)
qc.h(0)
qc.measure(0, 0)

sim = AerSimulator()
tqc = transpile(qc, sim)
result = sim.run(tqc, shots=1024).result()
print(result.get_counts())

{'1': 480, '0': 544}


In [6]:

from math import pi, asin, sqrt, cos, sin
import numpy as np
import random
from itertools import combinations
import sys

**Hamming distance**


In [7]:
#LA METTIAMO UN SECONDO DA PARTE
def hamming(a, b):
    return sum(int(ai) != int(bi) for ai, bi in zip(a, b))

def choose_by_inverse_hamming(measured_bits, valid_solutions):
    distances = [hamming(measured_bits, vs) for vs in valid_solutions]
    # avoid division by zero
    inv = [1.0/d if d > 0 else 0.0 for d in distances]
    s = sum(inv)
    if s == 0.0:
        return random.choice(valid_solutions)
    probs = [v/s for v in inv]
    chosen = random.choices(valid_solutions, weights=probs, k=1)[0]
    return list(chosen)

**Fitness**

In [8]:
#VA BENE
def fitness(x_bits, M):
    """Compute quadratic form x^T M x for binary vector x_bits and matrix M."""
    x = np.array(x_bits, dtype=float)
    return float(x @ M @ x.T)


**Check if the solution is valid**

In [9]:
#LA METTIAMO UN SECONDO DA PARTE
def all_valid_solutions(n, m, constrained=True):
    if not constrained:
        return [list(map(int, format(k, f'0{n}b'))) for k in range(2**n)]
    else:
        sols = []
        for combi in combinations(range(n), m):
            vec = [0]*n
            for i in combi:
                vec[i] = 1
            sols.append(vec)
        return sols

**matrix** **1**

In [10]:
M = np.array([[-0.269, 0.411, -0.079, 0.175],
              [0, -0.086, -0.222, -0.170],
              [0, 0, -0.463, 0.244],
              [0, 0, 0, -0.139]])

X = [1,1,0,0]

**matrix 2**

In [11]:
M = np.array([[0.430, -0.496, -0.443, 0.223],
              [0, 0.254, 0.029, -0.359],
              [0, 0, -0.424, -0.183],
              [0, 0, 0, 0.301]])

X = [1,0,0,1]

**matrix 3**

In [12]:
M = np.array([[-0.039, -0.327, 0.311, 0.100],
              [0, 0.364, 0.051, -0.387],
              [0, 0, 0.271, 0.116],
              [0, 0, 0, 0.261]])

X = [1,0,1,1]

**matrix 4**

> Aggiungi citazione



In [13]:
M = np.array([[-0.092, -0.425, 0.001, -0.116],
              [0, 0.167, -0.110, -0.370],
              [0, 0, 0.394, -0.061],
              [0, 0, 0, 0.104]])

X = [0,1,1,0]

**matrix** **5**

In [14]:
M = np.array([[0.409, -0.195, -0.248, 0.132],
              [0, -0.200, 0.242, -0.408],
              [0, 0, -0.205, 0.248],
              [0, 0, 0, -0.298]])

X = [1,0,0,0]

**funzione per fare 100 run per matrice**

input: matrices, solution, n_run = 100

def: qaco_run_main (ovvero il main del codice intero viene lanciata 100 volte)




In [ ]:
def multiple_runs(M, X, n_runs): ##
  #scorro sul vettore soluzione e conto gli 1's per impostare m
  m = 0
  for elem in X:
      if elem == 1:
          m = m + 1
  run = 1
  error_count = 0
  sum = 0
  for run in range(n_runs+1):
      print("Iteraton number: ", run)

      best_bits, iter_num = qaco_run_main(
                M,
                m,
                constrained=True
                )
      if X != best_bits:
          error_count += 1

      sum = sum + iter_num
  iter_mean = sum/n_runs
  print(f"Error probability: {(error_count/run)*100}%")
  print(f"Mean iterations: {iter_mean}")


**Main**

In [ ]:

from pickle import NONE
import numpy as np
import random
import matplotlib.pyplot as plt
import math
from math import pi, asin, sqrt, cos, sin

from math import pi, asin, sqrt, cos, sin

from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit import transpile
from numpy import pi





# Parametri QACO
#rho = 0.1                # Evaporazione del feromone, non serve
beta_e0 = 0.13             # Probabilità esplorazione iniziale
maxIter = 62             # Numero massimo di iterazioni
converCondition = 59      # Condizione di convergenza
n = 4                     # Dimensione matrice M, il codice è comunque generalizzato nella funzione update_thetas





#M = np.array([[-0.269, 0.411, -0.079, 0.175],
 #             [0, -0.086, -0.222, -0.170],
 #             [0, 0, -0.463, 0.244],
 #             [0, 0, 0, -0.139]])

# # Visualizzazione della matrice M
# print("Matrice M del problema:")
# print(M)

# -------------------------
# Lookup table (Table 1) and delta rule
# -------------------------
# Values given in Table 1 (multiples of pi). We'll store numeric values directly.
_lookup_table = {
    (0,0,True): -0.01 * pi,
    (0,0,False): 0.04 * pi,
    (0,1,True): -0.05 * pi,
    (0,1,False): 0.07 * pi,
    (1,0,True): 0.05 * pi,
    (1,0,False): -0.07 * pi,
    (1,1,True): 0.01 * pi,
    (1,1,False): -0.04 * pi             ## the condition of cos<0 is implemented later on
}


def init_pheromones(theta_ant):
    """Initialize RY angles θ_i = π/2 for all ant qubits."""
    theta_ant=[pi/2]*n #n = 4
    return theta_ant

def exploration_angle(beta_e):
    """Compute θ_e = 2 * arcsin(√β_e)."""
    return 2.0 * asin(sqrt(beta_e))

def calculate_fitness(X, M):
    X = np.array(X, dtype=float)
    return float(X @ M @ X.T)


def quantum_iteration_circuit_qiskit(theta_ant, theta_e, backend_shots=1, seed=None):

    #Creazione circuito quantistico
    n_local = len(theta_ant)  #4
    qc = QuantumCircuit(n_local + 1, n_local)   #1 control QUBIT (EXPLORATION), 4 target QUBIT (ANT)...  ; 5 qubits and 4 bits

    # (A) preparazione ant-qubit con RY(theta_i) su qubit 1..n
    for i in range(n_local):
      qc.ry(float(theta_ant[i]), i+1)

    # (B) per ogni ant-qubit: preparazione exploration RY(theta_e), CX(0,i+1), reset(0)---> see the appendix 2 about the reset
    for i in range(n_local):
      qc.ry(float(theta_e), 0)
      qc.cx(0, i+1)
      qc.reset(0)

    # (C) misura ant-qubit 1..n nei bit classici 0..n-1
    for i in range(n_local):
      qc.measure(i+1, i)

        # (D) esecuzione con AerSimulator
    sim = AerSimulator()
    t_qc = transpile(qc, sim)

    # se seed fornito, lo passiamo (seed_simulator è gestito da Aer.run kwargs)
    run_args = {"shots": int(backend_shots)}

    if seed is not None:   #####capire cosa sia il seed
                # seed_simulator e seed_transpiler sono argomenti utili per riproducibilità
        run_args["seed_simulator"] = int(seed)
        run_args["seed_transpiler"] = int(seed)

    job = sim.run(t_qc, **run_args)
    result = job.result()
    counts = result.get_counts()


    # invertiamo per ottenere ordine naturale q0..q_{n-1}

    #Prendi la bitstring più frequente e assicurati che sia stringa normale
    bitstring = max(counts, key=counts.get)
    if isinstance(bitstring, bytes):
        bitstring = bitstring.decode("utf-8")

    # Mantieni solo 0/1 e inverti (Qiskit restituisce l’ordine inverso dei qubit)
    bitstring = ''.join(ch for ch in bitstring if ch in ('0', '1'))
    measured_bits = [int(b) for b in bitstring[::-1]]

    #print("Bitstring misurata:", bitstring, "→", measured_bits)
    #print(f"Counts del circuito quantistico: {counts} \n")
    return measured_bits, counts



def lookup_delta_theta(xi, bi, is_current_better, theta_ant):
    """
    xi: bit misurato per qubit ANT i (0 o 1)
    bi: bit nella soluzione di riferimento b (0 o 1)
    is_current_better: True se f(x) > f(b), altrimenti False
    theta_i: angolo corrente del qubit ANT i

    Restituisce Δθ da sommare a θ_i.
    Applica la regola: se la riga “is_current_better == True” e cos(theta_i/2) < 0,
    inverte il segno di base. (come riportato nel paper)
    """
    key = (int(xi), int(bi), bool(is_current_better))
    base = _lookup_table[key]
    # righe “miglioramento” (is_current_better == True) sono soggette all’inversione
    if is_current_better and math.cos(theta_ant / 2) < 0:
        return -base
    return base

def update_thetas(theta_ant, measured_bits, best_bits, f_x, f_b):
    """
    Aggiorna in-place una lista di angoli theta_list per tutti i qubit i = 0..n-1.

    Parametri:
      - theta_ant: lista di valori θ_i correnti (float ANT qubit)
      - measured_bits: la soluzione misurata x come lista di 0/1
      - best_bits: la soluzione di confronto b (lista di 0/1), potrebbe essere None
      - f_x: fitness di x
      - f_b: fitness di b (se b è None, f_b può essere considerato -∞)

    Ritorna la nuova lista theta_ant aggiornata (o aggiorna in-place).
    """
    n = len(theta_ant)

    # Se best_bits è None (prima iterazione), definiamo bi = 0 per default

    for i in range(n):
        xi = measured_bits[i]
        bi = 0 if best_bits is None else best_bits[i]
        is_current_better = (f_x > f_b)
        delta = lookup_delta_theta(xi, bi, is_current_better, theta_ant[i])
        theta_new = theta_ant[i] + delta

        # Clip θ_i nell’intervallo [0, π]
        if theta_new < 0.0:
            theta_new = 0.0
        if theta_new > math.pi:
            theta_new = math.pi

        theta_ant[i] = theta_new

    return theta_ant

def qaco_run_main(M, m, constrained=True, beta_e0=0.13, seed=None, backend_shots=1024, verbose=False): ##############################################
    """
    Implementazione principale di QACO (Alg.2) per il caso unconstrained.
    Usa le funzioni:
        - quantum_iteration_circuit_qiskit(...)   # per costruire ed eseguire il circuito
        - update_thetas(...)                      # per aggiornare i theta (feromone)
    """
    n = M.shape[0]
    assert M.shape[1] == n
    if constrained:
        from math import comb
        nComb = comb(n, m) #number of all possible combinations between n and m
    else:
        nComb = 2**n


    #rng = np.random.default_rng(seed)
    n = M.shape[0]
    theta_ant=[0]*n
    best_bits=[0]*n
    # ---------------------------
    # Parametri iniziali
    # ---------------------------
    # θ_i iniziali = π/2

    theta_ant = init_pheromones(theta_ant)
    best_solution = None
    best_fitness = -np.inf

    print(f"--- QACO run (n={n},, m={m}) ---")
    print(f"converCondition = {converCondition}, maxIter = {maxIter}\n")

    # ---------------------------
    # Loop principale
    # ---------------------------

    for iter_num in range(1, maxIter + 1):
        # (1) Aggiorna beta_e e theta_e -> to discard suboptiml sols
        beta_e = beta_e0 + (1.0 - beta_e0) * (iter_num / maxIter)
        theta_e=exploration_angle(beta_e)

        # (2) Memorizza la best precedente
        f_prev_best = -np.inf if best_bits is None else calculate_fitness(best_bits, M)

        # (3) Esegui il circuito quantistico (singolo shot)
        measured_bits, counts = quantum_iteration_circuit_qiskit(theta_ant, theta_e, backend_shots=backend_shots, seed=seed)

        # (4) Calcola la fitness
        fitness=calculate_fitness(measured_bits, M)

        # (5) Aggiorna i θ_i con la lookup table
        update_thetas(theta_ant, measured_bits, best_bits, fitness, f_prev_best)

        # (6) Aggiorna best se necessario
        if fitness > best_fitness:
            best_fitness = fitness
            best_bits = measured_bits.copy()
            conver_counter = 0
        else:
            conver_counter += 1

        # (7) Condizione di convergenza
        if conver_counter >= converCondition:
            print("\n🟢 Convergenza raggiunta!")
            break
        # DA CAPIRE SE POSIZIONARLA QUI
        valid_solutions = all_valid_solutions(n, constrained, m)
         # If constrained problem and measured invalid -> generate alternative via inverse-hamming
        if constrained:
            if measured_bits not in valid_solutions:
                measured_bits = choose_by_inverse_hamming(measured_bits, valid_solutions)

        # (8 Stampa risultati
        if verbose:
            print(f"Iter {iter_num:3d}: measured={measured_bits} "
                  f"f_x={fitness:.5f} | best_f={best_fitness:.5f} "
                  f"conv={conver_counter}/{converCondition}")


    # ---------------------------
    # Risultato finale
    # ---------------------------
    print("\n=== RISULTATO FINALE ===")
    print("Best solution:", best_bits)
    print("Best fitness :", best_fitness)
    print("Iterations   :", iter_num)
    print("=========================\n")

    return best_bits, iter_num#{
  #      'best_solution': best_bits,
   #     'best_fitness': best_fitness,
     #   'iterations': iter_num,
    #    'converged': conver_counter >= converCondition
   # }

#for i in range(1, 5):
 # print(f"Constrained with m = {i}\n")
  #qaco_run_main(M, i, constrained=True)
multiple_runs(M, X, 100)

ModuleNotFoundError: No module named 'matplotlib'